In [3]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/My Drive/plant_pathology_kaggle")
import cv2
from keras.models import load_model
import numpy as np
import pandas as pd

sam_sub = pd.read_csv('sample_submission.csv')
#train_csv = pd.read_csv('train.csv')
test_csv = pd.read_csv('test.csv')

path = '/content/drive/My Drive/plant_pathology_kaggle/images/'
image_list = os.listdir(path)

test_images = test_csv["image_id"]
test_images = np.asarray(test_images) + ".jpg"

image_height = 150
image_width = 150
channels = 3

xtest = np.zeros((len(test_images), image_height, image_width, channels))

for i in range(len(test_images)):
  imgtest = test_images[i]
  img = cv2.imread('/content/drive/My Drive/plant_pathology_kaggle/images/'+imgtest)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (image_height, image_width))
  xtest[i] = img

xtest /= 255

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
net1 = load_model("/content/aug_train.h5")
net2 = load_model("/content/res_aug.h5")

p1 = net1.predict(xtest)
p2 = net2.predict(xtest)

predictions = (p1+p2)/2.0

In [0]:
sam_sub["healthy"].values[:] = 0
sam_sub["multiple_diseases"].values[:] = 0
sam_sub["rust"].values[:] = 0
sam_sub["scab"].values[:] = 0

for i in range(len(predictions)):
  sam_sub["scab"].values[i] = predictions[i, 0]
  sam_sub["multiple_diseases"].values[i] = predictions[i, 1]
  sam_sub["healthy"].values[i] = predictions[i, 2]
  sam_sub["rust"].values[i] = predictions[i, 3]

sam_sub.to_csv("/content/mixed.csv", index = False)